# Day 15

## Part 1

- Today's challenge details a hashing algorithm:
    - start with value = 0
    - value += ascii(char)
    - value *= 17
    - value %= 256
    - repeat for each char
- Find the sum of the hashed initialisation sequence values

In [1]:
from advent_of_code_utils.advent_of_code_utils import (
    parse_from_file, ParseConfig, markdown
)

parser = ParseConfig(',', str)

codes = parse_from_file('puzzle_input\\day_15.txt', parser)

In [3]:
def get_hash(code: str) -> int:
    """returns a byte value of the code string"""
    value = 0
    for char in code:
        value += ord(char)
        value *= 17
        value %= 256
    return value

example = [
    'rn=1','cm-','qp=3','cm=2','qp-','pc=4','ot=9','ab=5','pc-','pc=6','ot=7']

for code in example:
    print(code, get_hash(code))

rn=1 30
cm- 253
qp=3 97
cm=2 47
qp- 14
pc=4 180
ot=9 9
ab=5 197
pc- 48
pc=6 214
ot=7 231


In [4]:
# cool that works let's solve!
hash_sum = sum([get_hash(code) for code in codes])

markdown(
    '### Solution',
    f'The total hash sum is: {hash_sum}'
)

### Solution
The total hash sum is: 515210